# <center> UFCSPA - Informática Biomédica - Diagnóstico por Imagem (RAD0010)<center>
## <center> Trabalho Prático <center>
## <center> Sofia Faber Silveira <center>


#Conjunto de Dados
Kaggle - Skin Cancer: Malignant vs. Benign (https://www.kaggle.com/fanconic/skin-cancer-malignant-vs-benign)

# Imports

In [ ]:
import os
from PIL import Image

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import model_from_json

from sklearn.metrics import accuracy_score, confusion_matrix

import seaborn as sn

# Interface de Carregamento de Dados

In [ ]:
treino_benigno = 'drive/MyDrive/Diagnostico/data/train/benign'
treino_maligno = 'drive/MyDrive/Diagnostico/data/train/malignant'

teste_benigno = 'drive/MyDrive/Diagnostico/data/test/benign'
teste_maligno = 'drive/MyDrive/Diagnostico/data/test/malignant'

read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

# Baixa imagens de treino
benignos_treino = [read(os.path.join(treino_benigno, filename)) for filename in os.listdir(treino_benigno)]
imagens_benignos_treino = np.array(benignos_treino, dtype='uint8')
malignos_treino = [read(os.path.join(treino_maligno, filename)) for filename in os.listdir(treino_maligno)]
imagens_malignos_treino = np.array(malignos_treino, dtype='uint8')

# Baixa imagens de teste
benignos_teste = [read(os.path.join(teste_benigno, filename)) for filename in os.listdir(teste_benigno)]
imagens_benignos_teste = np.array(benignos_teste, dtype='uint8')
malignos_teste = [read(os.path.join(teste_maligno, filename)) for filename in os.listdir(teste_maligno)]
imagens_malignos_teste = np.array(malignos_teste, dtype='uint8')

# Tratamento de Dados

In [ ]:
# Labels de classificação: 0 - Benigno / 1 - Maligno
labels_benignos_treino = np.zeros(len(imagens_benignos_treino))
labels_malignos_treino = np.ones(len(imagens_malignos_treino))

labels_benignos_teste = np.zeros(len(imagens_benignos_teste))
labels_malignos_teste = np.ones(len(imagens_malignos_teste))

# Junta as imagens benignas e malignas
imagens_treino = np.concatenate((imagens_benignos_treino, imagens_malignos_treino), axis = 0)
labels_treino = np.concatenate((labels_benignos_treino, labels_malignos_treino), axis = 0)

imagens_teste = np.concatenate((imagens_benignos_teste, imagens_malignos_teste), axis = 0)
labels_teste = np.concatenate((labels_benignos_teste, labels_malignos_teste), axis = 0)

# Randomização das imagens
s = np.arange(len(imagens_treino))
np.random.shuffle(s)
imagens_treino = imagens_treino[s]
labels_treino = labels_treino[s]

s = np.arange(len(imagens_teste))
np.random.shuffle(s)
imagens_teste = imagens_teste[s]
labels_teste = labels_teste[s]

# Normalização das imagens
imagens_treino = imagens_treino/255.
imagens_teste = imagens_teste/255.

In [ ]:
imagens_treino.shape, imagens_teste.shape, labels_treino.shape, labels_teste.shape

# Exemplo de Dados

In [ ]:
# Plot de algumas imagens de treino com suas labels
fig = plt.figure(figsize=(15, 10))
fig.suptitle('Imagens Treino')
columns = 5
rows = 3

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if labels_treino[i] == 0:
        ax.title.set_text('Benigno')
    else:
        ax.title.set_text('Maligno')
    plt.imshow(imagens_treino[i])
plt.savefig('imagens_treino.png', format='png')
plt.show()

In [ ]:
# Plot de algumas imagens de teste com suas labels
fig = plt.figure(figsize=(15, 10))
fig.suptitle('Imagens Teste')
columns = 5
rows = 3

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if labels_teste[i] == 0:
        ax.title.set_text('Benigno')
    else:
        ax.title.set_text('Maligno')
    plt.imshow(imagens_teste[i])

plt.savefig('imagens_teste.png', format='png')
plt.show()

# Arquitetura da Rede Neural Convolucional

In [ ]:
def build():
    model = Sequential()

    # 1º Etapa
    # Convolução
    model.add(Conv2D(64, kernel_size=(7, 7), input_shape=(224,224,3), activation='relu')) 
    # Pooling
    model.add(MaxPool2D(pool_size = (2, 2)))
    # Dropout
    model.add(Dropout(0.25))

    # 2º Etapa
    # Convolução
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
    # Pooling
    model.add(MaxPool2D(pool_size = (2, 2)))
    # Dropout
    model.add(Dropout(0.25))

    # Flattening
    model.add(Flatten()) 
    
    # Full connection - Classificação
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid', name='predict'))
    
    model.summary()

    model.compile(optimizer = Adam(lr=1e-3) , loss = "binary_crossentropy", metrics = ["accuracy"])
    return model

# Metodologia de Treinamento e Obtenção do Modelo

In [ ]:
# Construção do Modelo
model = build()

# Visualização do Modelo
keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=False,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

In [ ]:
# Callbacks
learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', patience=5, verbose=1, factor=0.5, min_lr=1e-7)
best_model = ModelCheckpoint("best_model.h5", monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Treino do Modelo
history = model.fit(imagens_treino, labels_treino, validation_split=0.2, epochs=50, batch_size=64, callbacks=[learning_rate_reduction,best_model])

# Utilização do Melhor Modelo
model.load_weights("best_model.h5")

# Histórico do Treinamento
print(history.history.keys())

# Gráficos do Histórico de Treino

In [ ]:
# Plot do histórico de acurácia
plt.plot(history.history['accuracy'], color='#004D84')
plt.plot(history.history['val_accuracy'], color='#569ACB')
plt.title('Acurácia do Modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['Treino', 'Validação'], loc='upper left')
plt.savefig('acuracia.png', format='png')
plt.show()

# Plot do histórico de perda
plt.plot(history.history['loss'], color='#004D84')
plt.plot(history.history['val_loss'], color='#569ACB')
plt.title('Perda do Modelo')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.legend(['Treino', 'Validação'], loc='upper left')
plt.savefig('perda.png', format='png')
plt.show()

# Plot do histórico de acurácia de treino
plt.plot(history.history['accuracy'], label='Training', color='#004D84')
plt.title('Acurácia Treino')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.savefig('acuracia_treino.png', format='png')
plt.show()

# Plot do histórico de acurácia de validação
plt.plot(history.history["val_accuracy"], label='Validation', color='#004D84')
plt.title('Acurácia Validação')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.savefig('acuracia_validacao.png', format='png')
plt.show()

# Teste do Modelo

In [ ]:
# Avaliação do modelo com as imagens de teste
resultados = model.evaluate(imagens_teste, labels_teste, batch_size=64)
print('Evaluate: \n Perda:',resultados[0], '\n Acurácia:',(resultados[1]))

In [ ]:
# Predição das imagens de teste
resultados_predicao = model.predict(imagens_teste, batch_size=64)
labels_predicao = np.where(resultados_predicao>=0.5, 1,resultados_predicao) 
labels_predicao = np.where(labels_predicao<0.5, 0,labels_predicao)
print('Predict: \n Acurácia:',accuracy_score(labels_teste, labels_predicao))

# Classificação dos Dados

In [ ]:
# Classificações
print("Predição:\n", resultados_predicao)
print("Classificação Predição:\n", labels_predicao)
print("Classificação Correta:\n", labels_teste)

In [ ]:
# Matriz de confusão
matriz_confusao = confusion_matrix(labels_teste, labels_predicao)
print(matriz_confusao)

labels = ['benigno', 'maligno']
sn.heatmap(matriz_confusao, annot=True, cmap="Blues", fmt='d', xticklabels=(labels), yticklabels=(labels))

plt.title('Matriz de confusão')
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Classificação Predição')
plt.ylabel('Classificação Correta')
plt.savefig('matriz_confusao.png', format='png')
plt.show()

# Finalização

In [ ]:
# Salva modelo
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("best_model.h5")

In [ ]:
# Limpa memória
del model
K.clear_session()